In [1]:
#!pip -q install git+https://github.com/huggingface/transformers

In [2]:
#!pip install tiktoken chromadb langchain

In [3]:
# from git import Repo
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language

In [4]:
# Load
loader = GenericLoader.from_filesystem(
    "/home/ec2-user/SageMaker/VegRD_APD_Fruit_Phenotyping/scripts",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=5000),
)
documents = loader.load()
len(documents)

35

In [5]:
len(documents)

35

In [6]:
documents[0]

Document(page_content='import numpy as np\nimport os, sys\nfrom collections import defaultdict\nfrom get_data import get_images\nimport re\n\ndef update_config(config_defaults,**kwargs):\n    new_config = config_defaults.copy()\n    for key, value in kwargs.items():\n        if key in new_config:\n            new_config[key] = value\n        else:\n            raise KeyError(f"Unknown configuration key: {key}")\n    return new_config\n\n\ndef pattern_exists_in_string(string, pattern="BLK"):\n    if re.search(pattern, string):\n        return True\n    else:\n        return False\n\n\n# A dictionary of projects and images\nprojs = defaultdict(list)\n\nwith open("new_images.txt", "r") as f:\n    for l in f.readlines():\n        l=l.strip()\n        splitted= l.split("/")\n        projs[splitted[1]].append(l)\n\nfor k,v in projs.items():\n    get_images(k,s3_bucket="model-deployment-automation", files_to_copy=v)\n    if pattern_exists_in_string(k,pattern="BLK"):\n        from config impor

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, chunk_size=2000, chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)

199

In [8]:
#!pip install langchain sentence_transformers

In [9]:
#from langchain.embeddings import HuggingFaceInstructEmbeddings
#embeddings = HuggingFaceInstructEmbeddings(
#    query_instruction="Query about the code base: ",
#    model_kwargs = {'device': 'cpu'}
#)

In [10]:
#!pip install langchain sentence_transformers

from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
#from langchain.embeddings import HuggingFaceBgeEmbeddings

#model_name = "BAAI/BAAI/bge-base-en"
#model_kwargs = {"device": "cpu"}
#encode_kwargs = {"normalize_embeddings": True}
#hf = HuggingFaceBgeEmbeddings(
#    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
#)

In [12]:
#document_embeddings = [embeddings.embed_documents(text.page_content) for text in texts]
from langchain.vectorstores import Chroma
# Create a vector store
#vector_store = Chroma.from_embeddings(document_embeddings)
db = Chroma.from_documents(texts, embeddings)

In [13]:
#from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.vectorstores import Chroma

#db = Chroma.from_documents(texts, embeddings)
#retriever = db.as_retriever(
#    search_type="mmr",  # Also test "similarity"
#    search_kwargs={"k": 8},
#)

In [14]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory

In [15]:
#from langchain.llms.huggingface_pipeline import HuggingFacePipeline
#from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
#import torch

#device="cpu"

#model_id = "mistralai/Mistral-7B-Instruct-v0.1"
#tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype= torch.float32,device_map=device)
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000,temperature=0.1, do_sample=True)
#llm = HuggingFacePipeline(pipeline=pipe)


In [16]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

device="cuda"
model_id = "openchat/openchat-3.5-1210"
tokenizer = AutoTokenizer.from_pretrained(model_id,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype= "auto",device_map=device,trust_remote_code=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000)
llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [17]:
inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False).to("cuda")

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/transformers/generation/utils.py:1547: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<s> def print_prime(n):
   """
   Print all primes between 1 and n
   """
   for num in range(2, n+1):
      if is_prime(num):
         print(num)

def is_prime(n):
   """
   Check if n is a prime number
   """
   if n == 1:
      return False
   if n == 2:
      return True
   if n % 2 == 0:
      return False
   i = 3
   while i * i <= n:
      if n % i == 0:
         return False
      i += 2
   return True

print_prime(100)

# 2
# 3
# 5
# 7
# 11
# 13
# 17
# 


In [ ]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

gpu_chain = prompt | llm

question = "What is electroencephalography?"

print(gpu_chain.invoke({"question": question}))

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


In [19]:
#!pip install gpt4all 

In [20]:
#from langchain.llms import GPT4All

#llm = GPT4All(
#    model="gpt4all-falcon-q4_0.gguf",
#    max_tokens=2048,
#)

In [21]:
# To check the device and dtype
# Get the first parameter of the model and check its device and dtype
#first_param = next(model.parameters())
#model_device = first_param.device
#model_dtype = first_param.dtype

#print(f"Model is on device: {model_device}")
#print(f"Model data type: {model_dtype}")

In [22]:
## Test LLM without Context

#from langchain.prompts import PromptTemplate

#template = """Question: {question}

#Answer: Let's think step by step."""
#prompt = PromptTemplate.from_template(template)

#chain = prompt | llm

#question = "What is electroencephalography?"

#print(chain.invoke({"question": question}))

In [23]:
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=False)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(),memory=memory)

In [24]:
#!pip install langchainhub

In [25]:
#question = "How is curved backbone of hot peppers calculated?"
#docs = db.similarity_search(question)

In [26]:
#docs

In [27]:
#from langchain import hub

#rag_prompt = hub.pull("rlm/rag-prompt")
#from langchain.chains.question_answering import load_qa_chain

# Chain
#chain = load_qa_chain(llm, chain_type="stuff", prompt=rag_prompt)
#chain({"input_documents": docs, "question": question}, return_only_outputs=True)

In [28]:
#rag_prompt

In [29]:
question = "How is curved backbone of hot peppers calculated?"
result = qa(question)
result["answer"]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.00 GiB. GPU 0 has a total capacty of 15.77 GiB of which 683.62 MiB is free. Including non-PyTorch memory, this process has 15.10 GiB memory in use. Of the allocated memory 14.67 GiB is allocated by PyTorch, and 67.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
questions = [
    "How is curved backbone of hot peppers calculated?",
    "How to turn on/off visulaizations of phenotypes?",
    "How to turn on/off visulaizations of detections and phenotypes of hot peppers lsit all methods?",
]

for question in questions:
    result = qa(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")